In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import logging

from conda_forge_tick.utils import setup_logger

setup_logger(logging.getLogger("conda_forge_tick"), level="DEBUG")

In [8]:
from conda_forge_tick.make_graph import make_graph
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph

names = get_all_feedstocks(cached=True)

2023-03-01 06:28:20,317 INFO     conda_forge_tick.all_feedstocks || reading cached feedstocks


In [3]:
gx = load_graph(reset_bad=False)

gx = make_graph(names, gx, debug=True)

2023-03-01 06:22:57,767 INFO     conda_forge_tick.make_graph || reading graph
2023-03-01 06:22:57,769 INFO     conda_forge_tick.make_graph || start feedstock fetch loop
2023-03-01 06:22:58,204 ERROR    conda_forge_tick.feedstock_parser || Something odd happened when fetching feedstock tensorflow-gpu: 404
2023-03-01 06:22:58,211 INFO     conda_forge_tick.make_graph || feedstock fetch loop completed
2023-03-01 06:22:58,212 INFO     conda_forge_tick.make_graph || inferring nodes and edges
2023-03-01 06:23:31,949 INFO     conda_forge_tick.make_graph || new nodes and edges inferred
2023-03-01 06:23:31,950 INFO     conda_forge_tick.make_graph || memory usage: svmem(total=17179869184, available=6359851008, percent=63.0, used=9284153344, free=80265216, active=6266347520, inactive=5975015424, wired=3017805824)


In [4]:
gx.nodes["tensorflow-gpu"]["payload"].data

{'branch': 'main',
 'feedstock_name': 'tensorflow-gpu',
 'parsing_error': 'make_graph: 404',
 'pr_info': <conda_forge_tick.utils.LazyJson at 0x112710d60>,
 'version_pr_info': <conda_forge_tick.utils.LazyJson at 0x112710a90>}

In [9]:
def _parse_bad_attr(attrs: "AttrsTypedDict", not_bad_str_start: str) -> bool:
    """Overlook some bad entries"""
    bad = attrs.get("pr_info", {}).get("bad", False)
    if isinstance(bad, str):
        bad_bool = not bad.startswith(not_bad_str_start)
    else:
        bad_bool = bad

    return bad_bool or attrs.get("parsing_error", False)


import os
from conda_forge_tick.utils import LazyJson

noymls = []
for pth in os.listdir("node_attrs"):    
    lzj = LazyJson("node_attrs/" + pth)
    with lzj as attrs:
        if ("conda-forge.yml" not in attrs) and not attrs.get("archived", False) and pth[:-len(".json")] not in names:
            print("\n")
            print(pth)
            print(attrs.data)
            print("bad filter:", _parse_bad_attr(attrs, ""))
            
            noymls.append(pth)



proj.json
{'branch': 'master', 'feedstock_name': 'proj', 'pr_info': <conda_forge_tick.utils.LazyJson object at 0x144b75d60>, 'version_pr_info': <conda_forge_tick.utils.LazyJson object at 0x144b75670>}
bad filter: False


tensorflow-gpu.json
{'branch': 'main', 'feedstock_name': 'tensorflow-gpu', 'parsing_error': 'make_graph: 404', 'pr_info': <conda_forge_tick.utils.LazyJson object at 0x13abb3c70>, 'version_pr_info': <conda_forge_tick.utils.LazyJson object at 0x144b75640>}
bad filter: make_graph: 404


gfortran_impl_.json
{'feedstock_name': 'gfortran_impl_', 'pr_info': <conda_forge_tick.utils.LazyJson object at 0x117181be0>, 'version_pr_info': <conda_forge_tick.utils.LazyJson object at 0x144b75640>}
bad filter: False


proj4.json
{'branch': 'master', 'feedstock_name': 'proj4', 'pr_info': <conda_forge_tick.utils.LazyJson object at 0x117181be0>, 'time': 1543421436.145019, 'version_pr_info': <conda_forge_tick.utils.LazyJson object at 0x144b75700>}
bad filter: False


numpy-sugar.json
{'br

KeyboardInterrupt: 

In [ ]:
"blah".startswith("")

In [ ]:
lzj = LazyJson("node_attrs/proj.json")

In [ ]:
lzj.data

## Check Some Version Stuff

In [ ]:
from conda_forge_tick.update_upstream_versions import update_upstream_versions

In [ ]:
import copy

gxc = copy.deepcopy(gx)

for node in list(gxc.nodes):
    if node not in ["tzdata", "openssl", "jpeg", "cddlib"]:
        gxc.remove_node(node)

In [ ]:
update_upstream_versions(gxc, debug=True)

## Look at a Migration

In [ ]:
from conda_forge_tick.auto_tick import migration_factory, add_rebuild_broken_migrator

In [ ]:
mgs = []
add_rebuild_broken_migrator(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mmctx.effective_graph.nodes

In [ ]:
mg_name = "libffi33"

mgs = []
migration_factory(mgs, gx, only_keep=[mg_name])

for i in range(len(mgs)):
    if mgs[i].name == mg_name:
        break
mg = mgs[i]

In [ ]:
import copy

attrs = copy.deepcopy(mg.graph.nodes["python"]["payload"].data)

attrs["branch"] = "3.6"
mg.filter(attrs)

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)


In [ ]:
mmctx.effective_graph.nodes

## Check the Status Report

In [ ]:
from conda_forge_tick.status_report import  graph_migrator_status

In [ ]:
out2, build_sequence, gv = graph_migrator_status(mg, mg.graph)

In [ ]:
gv.view()

In [ ]:
build_sequence

In [ ]:
len(build_sequence)

In [ ]:
gx.nodes["proj"]["payload"].data